# Cyclic Groups and Generators

**Module 01** | 01-modular-arithmetic-groups

*Generators, multiplicative_order(), powers*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **cyclic groups and generators**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [Groups: A First Look](01c-groups-first-look.ipynb).
- Concepts and notation introduced in the previous notebook.

## Powers of an Element

In [ ]:
# Compute successive powers of g in (Z/7Z)*
R = Zmod(7)
g = R(3)
print([g^k for k in range(7)])

## Generators and Orders

In [ ]:
# Find the multiplicative order of each unit in Z/7Z
R = Zmod(7)
for x in R.list_of_elements_of_multiplicative_group():
    print(f'ord({x}) = {Mod(x,7).multiplicative_order()}')

## Identifying Generators

In [ ]:
# A generator has order equal to the group size
R = Zmod(7)
phi = euler_phi(7)
gens = [x for x in range(1,7) if Mod(x,7).multiplicative_order() == phi]
print('Generators:', gens)

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **cyclic groups and generators**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [Subgroups and Lagrange's Theorem](01e-subgroups-lagrange.ipynb)